In [1]:
import cv2
import torch
from ultralytics import YOLO

In [2]:
def natural_sort_key(filename):
    """Extract numerical values from filenames for correct sorting."""
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', filename)]

In [3]:
def detect_objects(model, image_path):
    image = cv2.imread(image_path)
    results = model(image)  # Perform object detection
    return results, image


In [4]:
def get_ceiling_floor_coordinates(results, floor_model):
    ceiling_y = None  # Initialize
    floor_y = None  # Initialize
    best_ceiling = None
    best_floor = None

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())  # Bounding box coords
            confidence = float(box.conf[0].cpu().numpy())  # Confidence score
            class_id = int(box.cls[0].cpu().numpy())  # Class ID
            
            class_name = floor_model.names.get(class_id, "Unknown")

            if class_name == "ceiling":
                if best_ceiling is None or confidence > best_ceiling[1]:
                    best_ceiling = ((x1, y1, x2, y2), confidence)

            if class_name == "floor":
                if best_floor is None or confidence > best_floor[1]:
                    best_floor = ((x1, y1, x2, y2), confidence)

    ceiling_mid, floor_mid = None, None  

    if best_ceiling:
        (_, ceiling_y1, _, ceiling_y2), _ = best_ceiling
        ceiling_y = (ceiling_y1 + ceiling_y2)// 2  # Midpoint

    if best_floor:
        (_, floor_y1, _, floor_y2), _ = best_floor
        floor_y = (floor_y1 + floor_y2)// 2  # Midpoint

    return ceiling_y, floor_y


In [5]:
def get_floor_coordinates(results):
    """Extract the top Y-coordinate and bounding box of the floor"""
    floor_y_top = None
    floor_box = None
    
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls.cpu().numpy().item())  # Get class ID
            x_min, y_min, x_max, y_max = map(int, box.xyxy[0])  # Bounding box
            
            if class_id == 1:  # Assuming class 1 is "floor"
                floor_y_top = y_min  # Top Y-coordinate of floor
                floor_box = (x_min, y_min, x_max, y_max)
    
    return floor_y_top, floor_box

In [6]:
def get_propeller_coordinates(results):
    """Extract the bottom Y-coordinates and bounding boxes of propellers"""
    propeller_boxes = []  # Store bounding boxes of detected propellers
    propeller_y_bottoms = []  # Store Y-bottom of propellers
    
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls.cpu().numpy().item())  # Get class ID
            x_min, y_min, x_max, y_max = map(int, box.xyxy[0])  # Bounding box
            
            if class_id == 0:  # Assuming class 0 is "propeller"
                propeller_y_bottoms.append(y_max)  # Store bottom Y-coordinate
                propeller_boxes.append((x_min, y_min, x_max, y_max))
    
    return propeller_y_bottoms, propeller_boxes

In [7]:
def detect_height_bottle(image_path, model):
    """Detect Coca-Cola bottles and return their bounding box height in pixels."""
    image = cv2.imread(image_path)
    results = model(image)
    coke_boxes = []
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = box.xyxy[0]  # Bounding box coordinates
            height_pixels = y2 - y1  # Bottle height in pixels
            real_height_cm = 26  # Known height of Coca-Cola bottle
            coke_boxes.append((x1, y1, x2, y2))
            pixel_to_cm_ratio = real_height_cm / height_pixels
            estimated_real_height = height_pixels * pixel_to_cm_ratio
            
            return height_pixels, estimated_real_height
    

    return None, None  # If no bottle is detected

In [8]:
def calculate_height(ceiling_y, floor_y, calibration_obj):
    """Calculate ceiling height using a calibration object (Coca-Cola bottle)."""
    if ceiling_y is None or floor_y is None:
        print("Error: Ceiling or Floor not detected!")
        return None  # Avoid TypeError

    pixel_height = floor_y - ceiling_y

    if calibration_obj:
        real_height, obj_pixel_height = calibration_obj
        pixel_to_cm_ratio = real_height / obj_pixel_height
        return pixel_height * pixel_to_cm_ratio
    
    return pixel_height  # Return pixel height if no calibration object


In [9]:
def draw_detections(image, propeller_boxes, floor_box, calibration_obj, floor_results):
    best_ceiling = None
    best_floor = None  # Ensure variables are initialized

    for i, (x1, y1, x2, y2) in enumerate(propeller_boxes):  # Unpacking tuple directly
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])  # Ensure integer values
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # Calculate height from floor to propeller
        pixel_height = y1 - floor_box[1]
        real_height = calculate_height(y1, floor_box[1], calibration_obj)
        height_text = f"Height: {real_height:.2f} cm"
        
        # Draw text label
        cv2.putText(image, height_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Label the calibration object
    if calibration_obj:
        real_height, obj_pixel_height = calibration_obj
        cal_text = f"Calibration: {real_height:.2f} cm ({obj_pixel_height} px)"
        cv2.putText(image, cal_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
    
    # Process results
    for result in floor_results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())  # Bounding box coordinates
            confidence = float(box.conf[0].cpu().numpy())  # Confidence score
            class_id = int(box.cls[0].cpu().numpy())  # Class ID

            # Get class name correctly
            class_name = floor_model.names.get(class_id, "Unknown")  # Ensure we get the correct class label

            # Store the most confident "ceiling" and "floor" detection
            if class_name == "ceiling":
                if best_ceiling is None or confidence > best_ceiling[1]:
                    best_ceiling = ((x1, y1, x2, y2), confidence, class_name)

            if class_name == "floor":
                if best_floor is None or confidence > best_floor[1]:
                    best_floor = ((x1, y1, x2, y2), confidence, class_name)

    # Draw only the best detections on the image
    for best in [best_ceiling, best_floor]:
        if best is not None:  # Ensure best_ceiling and best_floor are not None
            (x1, y1, x2, y2), confidence, class_name = best
            label = f'{class_name}: {confidence:.2f}'
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    return image


In [10]:
def draw_annotations(image, coke_boxes, ceiling_y, floor_y, pixel_to_cm_ratio=None):
    """Draws bounding boxes, vertical line, and labels on the image."""
    annotated_image = image.copy()
    
    # Draw Coca-Cola bounding boxes
    for (x1, y1, x2, y2, height_pixels) in coke_boxes:
        cv2.rectangle(annotated_image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box
        cv2.putText(annotated_image, f"{height_pixels}px", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Draw vertical line from floor to ceiling
    if ceiling_y is not None and floor_y is not None:
        mid_x = image.shape[1] // 2  # Center x-coordinate
        cv2.line(annotated_image, (mid_x, ceiling_y), (mid_x, floor_y), (0, 0, 255), 2)  # Red line
        
        # Calculate real-world ceiling height if calibration object exists
        pixel_height = floor_y - ceiling_y
        if pixel_to_cm_ratio:
            real_ceiling_height = pixel_height * pixel_to_cm_ratio
            height_text = f"Height: {real_ceiling_height:.2f} cm"
        else:
            height_text = f"Height: {pixel_height} pixels"

        # Add text label
        cv2.putText(annotated_image, height_text, (mid_x + 10, (ceiling_y + floor_y) // 2),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    return annotated_image

In [11]:
# import os
# import re
# import cv2
# import numpy as np

# def natural_sort_key(filename):
#     """Extract numerical values from filenames for correct sorting."""
#     return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', filename)]

# # Define folders
# image_folder = "Input Dataset folder"
# output_folder = "Output_test_folder_combined"
# propeller_output_folder = "Output_test_folder_propeller_only"

# os.makedirs(output_folder, exist_ok=True)
# os.makedirs(propeller_output_folder, exist_ok=True)

# # Load models
# coke_model = YOLO("best (8).pt")
# floor_model = YOLO("best (5).pt")
# propeller_model = YOLO("best (9).pt")



# ceil_heights = {}  # Store detected ceiling heights
# Propeller_heights = {}
# image_results = []

# # Get and sort image filenames
# image_files = sorted(
#     [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))],
#     key=natural_sort_key
# )

# i = 0
# for image_name in image_files:
#     image_path = os.path.join(image_folder, image_name)
    
#     # Detect Coca-Cola bottle height (for calibration)
#     height_pixels, estimated_real_height = detect_height_bottle(image_path, coke_model)
#     calibration_obj = (estimated_real_height, height_pixels) if height_pixels else None
    
#     # Detect ceiling and floor
#     floor_results, image = detect_objects(floor_model, image_path)
#     # Perform inference
        

#     # Store the best detection for ceiling and floor
#     # best_ceiling = None
#     # best_floor = None

#     # # Process results
#     # for result in floor_results:
#     #     for box in result.boxes:
#     #         x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())  # Bounding box coordinates
#     #         confidence = float(box.conf[0].cpu().numpy())  # Confidence score
#     #         class_id = int(box.cls[0].cpu().numpy())  # Class ID
                
#     #         # Get class name correctly
#     #         class_name = floor_model.names.get(class_id, "Unknown")  # Ensure we get the correct class label

#     #         # Store the most confident "ceiling" and "floor" detection
#     #         if class_name == "ceiling":
#     #             if best_ceiling is None or confidence > best_ceiling[1]:
#     #                 best_ceiling = ((x1, y1, x2, y2), confidence, class_name)

#     #         if class_name == "floor":
#     #             if best_floor is None or confidence > best_floor[1]:
#     #                 best_floor = ((x1, y1, x2, y2), confidence, class_name)

#     #     # Draw only the best detections on the image
#     # for best in [best_ceiling, best_floor]:
#     #     if best is not None:
#     #         (x1, y1, x2, y2), confidence, class_name = best
#     #         label = f'{class_name}: {confidence:.2f}'
#     #         cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
#     #         cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)



    
#     ceiling_y, floor_y = get_ceiling_floor_coordinates(floor_results,floor_model)
    
#     if ceiling_y is None or floor_y is None:
#         ceiling_height = "XXXXX"
#     elif calibration_obj:
#         ceiling_height = calculate_height(ceiling_y, floor_y, calibration_obj)
#     else:
#         ceiling_height = "XXXXX"
    
#     print(f"Ceiling Height for {image_name}: {ceiling_height}")
    
#     if ceiling_height != "XXXXX":
#         ceil_heights[image_name] = ceiling_height
    
#     # Read the image
#     image = cv2.imread(image_path)
    
#     # Detect propellers (AC)
#     propeller_results, _ = detect_objects(propeller_model, image_path)
#     propeller_y_bottoms, propeller_boxes = get_propeller_coordinates(propeller_results)

#     propeller_heights_details = []

#     propeller_heights_d = []
    
#     if propeller_boxes and calibration_obj:
#         for idx, propeller_y_bottom in enumerate(propeller_y_bottoms):
#             real_height = calculate_height(propeller_y_bottom, floor_y, calibration_obj)
#             propeller_heights_details.append(
#                 f"P{idx + 1}: {real_height:.2f} cm"
#             )
#             propeller_heights_d.append(real_height.item())

#     if propeller_heights_d != []:
#         Propeller_heights[image_name] = propeller_heights_d

    

    
#     # if propeller_boxes and calibration_obj:
#     #     propeller_heights = [f"{calculate_height(y, floor_y, calibration_obj):.2f} cm" for y in propeller_y_bottoms]
#     # else:
#     #     propeller_heights = []
    
#     ac_height = ", ".join(propeller_heights_details) if propeller_heights_details else "YYYY"
#     print(f"AC Height for {image_name}: {ac_height}")

#     # ac_height = "\n".join(propeller_heights_details) if propeller_heights_details else "YYYY"
#     # print(f"AC Height for {image_name}:\n{ac_height}")
    
    
#     # Draw vertical reference line
#     if ceiling_y is not None and floor_y is not None and ceiling_height != "XXXXX":
#         mid_x = image.shape[1] // 2
#         cv2.line(image, (mid_x, ceiling_y), (mid_x, floor_y), (0, 255, 0), 3)
#         text_position = (mid_x + 10, (ceiling_y + floor_y) // 2)
#         cv2.putText(image, f"{ceiling_height:.2f} cm", text_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    



#     # If propellers are detected, save in a separate folder
#     # if propeller_boxes:
#     #     propeller_output_path = os.path.join(propeller_output_folder, image_name)
#     #     cv2.imwrite(propeller_output_path, image)
#     #     print(f"Propeller Output saved at: {propeller_output_path}")

#     # Draw propeller detections
#     if calibration_obj:
#         for idx, (box, real_height) in enumerate(zip(propeller_boxes, propeller_heights_d)):
#             x1, y1, x2, y2 = box
#             label = f"P{idx + 1}: {real_height:.2f} cm"
#             cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw bounding box
#             cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)  # Draw correct label
    
#         output_path = os.path.join(output_folder, image_name)
#         cv2.imwrite(output_path, image)
#         print(f"Output saved at: {output_path}")

#         # If propellers are detected, save in a separate folder
#         if propeller_boxes:
#             propeller_output_path = os.path.join(propeller_output_folder, image_name)
#             cv2.imwrite(propeller_output_path, image)
#             print(f"Propeller Output saved at: {propeller_output_path}")
        
#     # # Draw propeller detections
#     # if calibration_obj:
#     #     output_image = draw_detections(image, propeller_boxes, (floor_y, floor_y), calibration_obj,floor_results)
#     #     output_path = os.path.join(output_folder, image_name)
#     #     cv2.imwrite(output_path, output_image)
#     #     print(f"Output saved at: {output_path}")

#     #     # If propellers are detected, save in a separate folder
#     #     if propeller_boxes:
#     #         propeller_output_path = os.path.join(propeller_output_folder, image_name)
#     #         cv2.imwrite(propeller_output_path, output_image)
#     #         print(f"Propeller Output saved at: {propeller_output_path}")
    
#     # Store results
#     image_results.append(f"Photo {i+1} | ID {image_name} | Ceiling height {ceiling_height} | AC Height {ac_height}")
#     i += 1

# # Print final results
# print("\nFinal Results:")
# for result in image_results:
#     print(result)


In [12]:
# import os
# import re
# import cv2
# import numpy as np
# from ultralytics import YOLO

# def natural_sort_key(filename):
#     """Extract numerical values from filenames for correct sorting."""
#     return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', filename)]

# # Define folders
# image_folder = "Input Dataset folder"
# output_folder = "Output_test_folder_combined"
# propeller_output_folder = "Output_test_folder_propeller_only"

# os.makedirs(output_folder, exist_ok=True)
# os.makedirs(propeller_output_folder, exist_ok=True)

# # Load models
# coke_model = YOLO("best (8).pt")
# floor_model = YOLO("best (5).pt")
# propeller_model = YOLO("best (9).pt")

# ceil_heights = {}  # Store detected ceiling heights
# Propeller_heights = {}
# image_results = []

# # Get and sort image filenames
# image_files = sorted(
#     [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))],
#     key=natural_sort_key
# )

# i = 0
# for image_name in image_files:
#     if not image_name.lower().endswith(('.jpg', '.jpeg', '.png')):
#         continue
    
#     image_path = os.path.join(image_folder, image_name)

#     # Detect Coca-Cola bottle height (for calibration)
#     height_pixels, estimated_real_height = detect_height_bottle(image_path, coke_model)
#     calibration_obj = (estimated_real_height, height_pixels) if height_pixels else None
    
#     # Detect ceiling and floor
#     floor_results, image = detect_objects(floor_model, image_path)
#     ceiling_y, floor_y = get_ceiling_floor_coordinates(floor_results, floor_model)
#     coke_detected = height_pixels is not None

#     if ceiling_y is None or floor_y is None:
#         ceiling_height = "XXXXX"
#     elif calibration_obj:
#         ceiling_height = calculate_height(ceiling_y, floor_y, calibration_obj)
#     else:
#         ceiling_height = "XXXXX"
    
#     print(f"Ceiling Height for {image_name}: {ceiling_height}")
    
#     if ceiling_height != "XXXXX":
#         ceil_heights[image_name] = ceiling_height
    
#     # Detect propellers (AC)
#     propeller_results, _ = detect_objects(propeller_model, image_path)
#     propeller_y_bottoms, propeller_boxes = get_propeller_coordinates(propeller_results)

#     propeller_heights_details = []
#     propeller_heights = []
#     propeller_heights_d = []
    
#     if propeller_boxes and calibration_obj:
#         for idx, propeller_y_bottom in enumerate(propeller_y_bottoms):
#             real_height = calculate_height(propeller_y_bottom, floor_y, calibration_obj)
#             propeller_heights_details.append(
#                 f"P{idx + 1}: {real_height:.2f} cm"
#             )
#             propeller_heights_d.append(real_height.item())

#     if propeller_heights_d != []:
#         Propeller_heights[image_name] = propeller_heights_d

#     ac_height = ", ".join(propeller_heights_details) if propeller_heights_details else "YYYY"
#     print(f"AC Height for {image_name}: {ac_height}")

#     # ac_height = "\n".join(propeller_heights_details) if propeller_heights_details else "YYYY"
#     # print(f"AC Height for {image_name}:\n{ac_height}")
    
    
#     # Draw vertical reference line
#     if ceiling_y is not None and floor_y is not None and ceiling_height != "XXXXX":
#         mid_x = image.shape[1] // 2
#         cv2.line(image, (mid_x, ceiling_y), (mid_x, floor_y), (0, 255, 0), 3)
#         text_position = (mid_x + 10, (ceiling_y + floor_y) // 2)
#         cv2.putText(image, f"{ceiling_height:.2f} cm", text_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    



#     # If propellers are detected, save in a separate folder
#     # if propeller_boxes:
#     #     propeller_output_path = os.path.join(propeller_output_folder, image_name)
#     #     cv2.imwrite(propeller_output_path, image)
#     #     print(f"Propeller Output saved at: {propeller_output_path}")

#     # Draw propeller detections
#     if calibration_obj:
#         for idx, (box, real_height) in enumerate(zip(propeller_boxes, propeller_heights_d)):
#             x1, y1, x2, y2 = box
#             label = f"P{idx + 1}: {real_height:.2f} cm"
#             cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw bounding box
#             cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)  # Draw correct label
    
#         output_path = os.path.join(output_folder, image_name)
#         cv2.imwrite(output_path, image)
#         print(f"Output saved at: {output_path}")

#         # If propellers are detected, save in a separate folder
#         if propeller_boxes:
#             propeller_output_path = os.path.join(propeller_output_folder, image_name)
#             cv2.imwrite(propeller_output_path, image)
#             print(f"Propeller Output saved at: {propeller_output_path}")
        
#     # # Draw propeller detections
#     # if calibration_obj:
#     #     output_image = draw_detections(image, propeller_boxes, (floor_y, floor_y), calibration_obj,floor_results)
#     #     output_path = os.path.join(output_folder, image_name)
#     #     cv2.imwrite(output_path, output_image)
#     #     print(f"Output saved at: {output_path}")

#     #     # If propellers are detected, save in a separate folder
#     #     if propeller_boxes:
#     #         propeller_output_path = os.path.join(propeller_output_folder, image_name)
#     #         cv2.imwrite(propeller_output_path, output_image)
#     #         print(f"Propeller Output saved at: {propeller_output_path}")

#     # Determine the new filename
#     if not coke_detected and not propeller_boxes:
#         new_filename = f"{image_name.split('.')[0]}-2-0-0-0.jpg"
#     elif coke_detected and not propeller_boxes:
#         new_filename = f"{image_name.split('.')[0]}-1-0-0-0.jpg"
#     elif not coke_detected and propeller_boxes:
#         new_filename = f"{image_name.split('.')[0]}-0-0-0-0.jpg"
#     else:  # Both coke bottle and propeller detected
#         propeller_height_cm = max(propeller_heights) if propeller_heights else 0
#         new_filename = f"{image_name.split('.')[0]}-1-0-0-{int(propeller_height_cm)}.jpg"
    
#     # # Save the image with the new filename
#     # output_path = os.path.join(output_folder, new_filename)
#     # cv2.imwrite(output_path, image)
#     # print(f"Output saved at: {output_path}")


#     # Store results
#     image_results.append(f"Photo {i+1} | ID {new_filename} | Ceiling height {ceiling_height} | AC Height {ac_height}")
#     # i += 1



    
#     # Determine the new filename
#     if not coke_detected and not propeller_boxes:
#         new_filename = f"{image_name.split('.')[0]}-2-0-0-0.jpg"
#     elif coke_detected and not propeller_boxes:
#         new_filename = f"{image_name.split('.')[0]}-1-0-0-0.jpg"
#     elif not coke_detected and propeller_boxes:
#         new_filename = f"{image_name.split('.')[0]}-0-0-0-0.jpg"
#     else:  # Both coke bottle and propeller detected
#         propeller_height_cm = max(propeller_heights) if propeller_heights else 0
#         new_filename = f"{image_name.split('.')[0]}-1-0-0-{int(propeller_height_cm)}.jpg"

# # Draw propeller detections
#     if calibration_obj:
#         for idx, (box, real_height) in enumerate(zip(propeller_boxes, propeller_heights_d)):
#             x1, y1, x2, y2 = box
#             label = f"P{idx + 1}: {real_height:.2f} cm"
#             cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw bounding box
#             cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)  # Draw correct label
    
#         output_path = os.path.join(output_folder, new_filename)
#         cv2.imwrite(output_path, image)
#         print(f"Output saved at: {output_path}")

#         # If propellers are detected, save in a separate folder
#         if propeller_boxes:
#             propeller_output_path = os.path.join(propeller_output_folder, image_name)
#             cv2.imwrite(propeller_output_path, image)
#             print(f"Propeller Output saved at: {propeller_output_path}")
#     i += 1


    
# # Print final results
# print("\nFinal Results:")
# for result in image_results:
#     print(result)


In [ ]:
def detect_and_measure_propellers(image_path, box_model, round_model, pixel_to_cm_ratio):
    """Detect and measure both round and box propellers in the image."""
    box_results, image = detect_objects(box_model, image_path)
    round_results, _ = detect_objects(round_model, image_path)
    
    measurements = []
    CONFIDENCE_THRESHOLD = 0.58
    
    # Process box propellers
    for result in box_results:
        for i, box in enumerate(result.boxes.xyxy):
            confidence = float(result.boxes.conf[i])
            if confidence < CONFIDENCE_THRESHOLD:
                continue
            x1, y1, x2, y2 = map(int, box.tolist())
            width_px = x2 - x1
            height_px = y2 - y1
            width_cm = width_px * pixel_to_cm_ratio if pixel_to_cm_ratio else "Unknown"
            height_cm = height_px * pixel_to_cm_ratio if pixel_to_cm_ratio else "Unknown"
            measurements.append(("Box", width_cm, height_cm))
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            label = f"W:{width_cm:.2f} cm, H:{height_cm:.2f}cm" if pixel_to_cm_ratio else "Box: Unknown"
            cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    
    # Process round propellers
    for result in round_results:
        for i, box in enumerate(result.boxes.xyxy):
            confidence = float(result.boxes.conf[i])
            if confidence < CONFIDENCE_THRESHOLD:
                continue
            x1, y1, x2, y2 = map(int, box.tolist())
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2
            radius_px = max((x2 - x1), (y2 - y1)) // 2
            diameter_px = 2 * radius_px
            diameter_cm = diameter_px * pixel_to_cm_ratio if pixel_to_cm_ratio else "Unknown"
            circumference_cm = np.pi * diameter_cm if pixel_to_cm_ratio else "Unknown"
            measurements.append(("Round", diameter_cm, circumference_cm))
            cv2.circle(image, (center_x, center_y), radius_px, (0, 0, 255), 2)
            label = f"C:{circumference_cm:.2f}cm" if pixel_to_cm_ratio else "Round: Unknown"
            cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    return measurements, image


In [13]:
import os
import re
import cv2
import numpy as np
from ultralytics import YOLO

def natural_sort_key(filename):
    """Extract numerical values from filenames for correct sorting."""
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', filename)]

# Define folders
image_folder = "Input Dataset folder"
output_folder = "Output_test_folder_combined"
propeller_output_folder = "Output_test_folder_propeller_only"

os.makedirs(output_folder, exist_ok=True)
os.makedirs(propeller_output_folder, exist_ok=True)

# Load models
coke_model = YOLO("best (8).pt")
floor_model = YOLO("best (5).pt")
propeller_model = YOLO("best (9).pt")

ceil_heights = {}  # Store detected ceiling heights
propeller_heights = {}
image_results = []
txt_output_path = os.path.join(output_folder, "detection_results.txt")

# Get and sort image filenames
image_files = sorted(
    [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))],
    key=natural_sort_key
)

with open(txt_output_path, "w") as txt_file:
    for i, image_name in enumerate(image_files):
        image_path = os.path.join(image_folder, image_name)

        # Detect Coca-Cola bottle height (for calibration)
        height_pixels, estimated_real_height = detect_height_bottle(image_path, coke_model)
        calibration_obj = (estimated_real_height, height_pixels) if height_pixels else None
        coke_detected = height_pixels is not None

        # Detect ceiling and floor
        floor_results, image = detect_objects(floor_model, image_path)
        ceiling_y, floor_y = get_ceiling_floor_coordinates(floor_results, floor_model)

        if ceiling_y is None or floor_y is None:
            ceiling_height = "XXXXX"
        elif calibration_obj:
            ceiling_height = calculate_height(ceiling_y, floor_y, calibration_obj)
        else:
            ceiling_height = "XXXXX"

        if ceiling_height != "XXXXX":
            ceil_heights[image_name] = ceiling_height

        # Detect propellers (AC)
        propeller_results, _ = detect_objects(propeller_model, image_path)
        propeller_y_bottoms, propeller_boxes = get_propeller_coordinates(propeller_results)

        propeller_heights_details = []
        propeller_heights_d = []
        
        if propeller_boxes and calibration_obj:
            for idx, propeller_y_bottom in enumerate(propeller_y_bottoms):
                real_height = calculate_height(propeller_y_bottom, floor_y, calibration_obj)
                propeller_heights_details.append(f"P{idx + 1}: {real_height:.2f} cm")
                propeller_heights_d.append(real_height.item())

        if propeller_heights_d:
            propeller_heights[image_name] = propeller_heights_d

        ac_height = ", ".join(propeller_heights_details) if propeller_heights_details else "YYYY"

            # Draw vertical reference line
        if ceiling_y is not None and floor_y is not None and ceiling_height != "XXXXX":
            mid_x = image.shape[1] // 2
            cv2.line(image, (mid_x, ceiling_y), (mid_x, floor_y), (0, 255, 0), 3)
            text_position = (mid_x + 10, (ceiling_y + floor_y) // 2)
            cv2.putText(image, f"{ceiling_height:.2f} cm", text_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    



    # If propellers are detected, save in a separate folder
    # if propeller_boxes:
    #     propeller_output_path = os.path.join(propeller_output_folder, image_name)
    #     cv2.imwrite(propeller_output_path, image)
    #     print(f"Propeller Output saved at: {propeller_output_path}")

    # Draw propeller detections
        if calibration_obj:
            for idx, (box, real_height) in enumerate(zip(propeller_boxes, propeller_heights_d)):
                
                x1, y1, x2, y2 = box
                label = f"P{idx + 1}: {real_height:.2f} cm"
                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw bounding box
                cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)  # Draw correct label
    
                output_path = os.path.join(output_folder, image_name)
                cv2.imwrite(output_path, image)
                print(f"Output saved at: {output_path}")

        # If propellers are detected, save in a separate folder
        if propeller_boxes:
            propeller_output_path = os.path.join(propeller_output_folder, image_name)
            cv2.imwrite(propeller_output_path, image)
            print(f"Propeller Output saved at: {propeller_output_path}")

        # Determine the new filename
        new_filename = f"{image_name.split('.')[0]}-1-0-0-0.jpg" if coke_detected else f"{image_name.split('.')[0]}-2-0-0-0.jpg"
        if propeller_boxes:
            propeller_height_cm = max(propeller_heights_d) if propeller_heights_d else 0
            new_filename = f"{image_name.split('.')[0]}-1-0-0-{int(propeller_height_cm)}.jpg"
        
        # Save the image with the modified filename (ensuring only 55 images)
        output_path = os.path.join(output_folder, new_filename)
        cv2.imwrite(output_path, image)
        print(f"Output saved at: {output_path}")

        # Save only propeller images separately
        if propeller_boxes:
            propeller_output_path = os.path.join(propeller_output_folder, new_filename)
            cv2.imwrite(propeller_output_path, image)
            print(f"Propeller Output saved at: {propeller_output_path}")

        # Write only the new filenames to the text file
        txt_file.write(new_filename + "\n")

        # Store results for final output
        image_results.append(f"Photo {i+1} | ID {new_filename} | Ceiling height {ceiling_height} | AC Height {ac_height}")

# Print final results
print("\nFinal Results:")
for result in image_results:
    print(result)

print(f"\nDetection results saved at: {txt_output_path}")



0: 800x608 1 750ml coke, 29.6ms
Speed: 2.2ms preprocess, 29.6ms inference, 59.7ms postprocess per image at shape (1, 3, 800, 608)

0: 640x480 2 ceilings, 2 floors, 4 wallss, 38.8ms
Speed: 1.0ms preprocess, 38.8ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 480)

0: 800x608 (no detections), 7.3ms
Speed: 1.8ms preprocess, 7.3ms inference, 0.3ms postprocess per image at shape (1, 3, 800, 608)
Output saved at: Output_test_folder_combined/imageA-1-0-0-0.jpg

0: 800x608 1 750ml coke, 6.8ms
Speed: 1.9ms preprocess, 6.8ms inference, 0.8ms postprocess per image at shape (1, 3, 800, 608)

0: 640x480 4 ceilings, 2 floors, 4 wallss, 6.2ms
Speed: 1.1ms preprocess, 6.2ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 480)

0: 800x608 (no detections), 6.9ms
Speed: 2.1ms preprocess, 6.9ms inference, 0.4ms postprocess per image at shape (1, 3, 800, 608)
Output saved at: Output_test_folder_combined/imageAA-1-0-0-0.jpg

0: 800x608 1 750ml coke, 6.9ms
Speed: 2.0ms preprocess,

In [14]:
# import os
# import string

# def number_to_alphabet(n):
#     """Convert number to alphabetic sequence like A, B, ..., Z, AA, AB, ..."""
#     result = ""
#     while n >= 0:
#         result = chr(n % 26 + ord('A')) + result
#         n = n // 26 - 1
#     return result

# # Define folder
# image_folder = "Input Dataset folder"

# # Get and sort image filenames
# image_files = sorted(
#     [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))],
#     key=lambda x: int(''.join(filter(str.isdigit, x)))  # Extract numbers and sort
# )

# # Rename files
# for index, image_name in enumerate(image_files):
#     old_path = os.path.join(image_folder, image_name)
#     new_name = f"image{number_to_alphabet(index)}{os.path.splitext(image_name)[1]}"  
#     new_path = os.path.join(image_folder, new_name)
    
#     os.rename(old_path, new_path)
#     print(f"Renamed: {image_name} -> {new_name}")

# print("Renaming completed successfully!")

In [15]:
txt_output_path = os.path.join(output_folder, "detection_results.txt")

with open(txt_output_path, "w") as txt_file:
    for image_name in image_files:
        # Define the new filename based on detection results
        if coke_detected and not propeller_boxes:
            new_filename = f"{image_name.split('.')[0]}-1-0-0-0.jpg"
        elif not coke_detected and propeller_boxes:
            new_filename = f"{image_name.split('.')[0]}-0-0-0-0.jpg"
        elif not coke_detected and not propeller_boxes:
            new_filename = f"{image_name.split('.')[0]}-2-0-0-0.jpg"
        else:  # Both Coca-Cola bottle and propeller detected
            propeller_height_cm = max(propeller_heights_d) if propeller_heights_d else 0
            new_filename = f"{image_name.split('.')[0]}-1-0-0-{int(propeller_height_cm)}.jpg"

        # Write the formatted filename to the text file
        txt_file.write(new_filename + "\n")

print(f"Detection results saved at: {txt_output_path}")


Detection results saved at: Output_test_folder_combined/detection_results.txt
